In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
import gradio as gr

In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key=os.getenv("GOOGLE_API_KEY")
anthropic_api_key=os.getenv("ANTHROPIC_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API key not set")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyBa
Anthropic API Key exists and begins sk-ant-


In [4]:
openai=OpenAI()

anthropic_url = "https://api.anthropic.com/v1"
gemini_url="https://generativelanguage.googleapis.com/v1beta/openai/"

anthropic=OpenAI(base_url=anthropic_url,api_key=anthropic_api_key)
gemini=OpenAI(base_url=gemini_url,api_key=google_api_key)

In [5]:
system_message="You are a helpful Assistant"

def message_gpt(prompt):
    messages=[{"role":"system","content":system_message},{"role":"user","content":prompt}]
    response=openai.chat.completions.create(model="gpt-4.1-mini",messages=messages)
    return response.choices[0].message.content

In [6]:
message_gpt("What is today's date?")

"Today's date is April 27, 2024."

### User Interface

In [8]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [9]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [ ]:
gr.Interface(fn=shout,inputs="textbox",outputs="textbox",flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input Hello
Shout has been called with input Hello


In [12]:
gr.Interface(fn=shout,inputs="textbox",outputs="textbox",flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input teee


### Authentication

In [13]:
gr.Interface(fn=shout,inputs="textbox",outputs="textbox",flagging_mode="never").launch(inbrowser=True,auth=("nv","oranges"))

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input checking


In [ ]:
message_input=gr.Textbox(label="your message:",info="Enter a message to be shouted",lines=7)
message_output=gr.Textbox(label="Response:",lines=8)
view=gr.Interface(
    fn=shout,
    inputs=[message_input],
    outputs=[message_output],
    examples=["hello","howdy"],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input checking
Shout has been called with input hello


In [ ]:
message_input=gr.Textbox(label="Your Message:",info="Enter a message for GPT-4.1-mini", lines=7)
message_output=gr.Textbox(label="Response:",lines=8)

view=gr.Interface(
    fn=shout,
    inputs=[message_input],
    outputs=[message_output],
    examples=["hello","howdy"],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input sdhfsdhf


In [20]:
system_message="You are a helpful assistant that responds in markdown without code blocks"

message_input = gr.Textbox(label="Your Message",info="Enter a message for GPT-4.1-mini", lines=7)
message_output=gr.Markdown(label="Response:")
view=gr.Interface(
    fn=message_gpt,
    title="GPT",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI enginer"
    ],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [21]:
def stream_gpt(prompt):
    messages=[
        {"role":"system","content":system_message},
        {"role":"user","content":prompt}
    ]
    stream=openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        stream=True
    )
    result=""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [22]:
system_message="You are a helpful assistant that responds in markdown without code blocks"

message_input = gr.Textbox(label="Your Message",info="Enter a message for GPT-4.1-mini", lines=7)
message_output=gr.Markdown(label="Response:")
view=gr.Interface(
    fn=stream_gpt,
    title="GPT",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI enginer"
    ],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


In [23]:
def stream_claude(prompt):
    messages=[
        {"role":"system","content":system_message},
        {"role":"user","content":prompt}
    ]
    stream = anthropic.chat.completions.create(
        model="claude-sonnet-4-5-20250929",
        messages=messages,
        stream=True
    )
    result=""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
system_message="You are a helpful assistant that responds in markdown without code blocks"

message_input = gr.Textbox(label="Your Message",info="Enter a message for GPT-4.1-mini", lines=7)
message_output=gr.Markdown(label="Response:")
view=gr.Interface(
    fn=stream_claude,
    title="GPT",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI enginer"
    ],
    flagging_mode="never"
)
view.launch()

In [25]:
def stream_model(prompt,model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [27]:

message_input = gr.Textbox(label="Your message:", info="ENter a message for the LLM", lines=7)
model_selector = gr.Dropdown(["GPT","Claude"], label="select model", value="GPT")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_model,
    title="LLMs",
    inputs=[message_input,model_selector],
    outputs=[message_output],
    examples=[
        ["Explain the Transformer architecture to a layperson","GPT"],
        ["Explain the Transformer architecture to an aspiring AI engineer","Claude"]
    ],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


In [28]:
system_message=""" 
You are an assistant that analyzes the contents of a company website landing page
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
"""

In [34]:
from scraper import fetch_website_contents

In [37]:
def stream_brochure(company_name,url,model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page: \n"
    prompt += fetch_website_contents(url)
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result
    

In [38]:
name_input = gr.Textbox(label="Company name:")
url_input = gr.Textbox(label="Landing Page URL including http:// or https://")
model_selector = gr.Dropdown(["GPT","Claude"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_brochure,
    title="Brochure Generator",
    inputs=[name_input,url_input,model_selector],
    outputs=[message_output],
    examples=[
        ["Huggingface","https://huggingface.co","GPT"],
        ["SMX","https://smxtech.com","Claude"]
    ],
    flagging_mode = "never"

)
view.launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.
